In [1]:
#Import python libraries
import dvc.api
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os

In [4]:
# import requiered module
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from load_data import LoadData
from preprocessing import Preprocess
from plot_data import PlotData

In [48]:
loader = LoadData()
preprocess = Preprocess()
plot = PlotData()

In [58]:
train_path = 'data/train_merged.csv'
test_path = 'data/test_merged.csv'
repo = 'https://github.com/yonamg/Pharmaceutical-Sales-Prediction'
tra_ver = 'a09c75a'
tst_ver = '95939a9'

In [59]:
train_url = dvc.api.get_url(
    path=train_path,
    repo=repo,
    rev=tra_ver
)
print(train_url)

C:\.dvc\tmp\pharma_data\c8\d22784adf34e32c0fb517a150795ed


In [60]:
# read csv files 
train_df = loader.read_csv(train_url)

In [61]:
test_url = dvc.api.get_url(
    path=test_path,
    repo=repo,
    rev=tst_ver
)
print(test_url)

C:\.dvc\tmp\pharma_data\ac\072eb0cdbb58a154296f8eb4152f5f


In [62]:
# read csv files 
test_df = loader.read_csv(test_url)

In [63]:
train_df.sort_values(["Store","Date"], ignore_index=True, inplace=True)
test_df.sort_values(["Store","Date"], ignore_index=True, inplace=True)

Set data as index

In [64]:
train_df.set_index('Date', inplace=True)
test_df.set_index('Date', inplace=True)

In [65]:
train_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,weekday,weekofyear,weekend
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1,1,0,0,0,0,a,1,c,a,...,0,NaN,NaN,NaN,2013,1,1,1,1,0
2013-01-02,1,2,5530,668,1,0,0,1,c,a,...,0,NaN,NaN,NaN,2013,1,2,2,1,0
2013-01-03,1,3,4327,578,1,0,0,1,c,a,...,0,NaN,NaN,NaN,2013,1,3,3,1,0
2013-01-04,1,4,4486,619,1,0,0,1,c,a,...,0,NaN,NaN,NaN,2013,1,4,4,1,0
2013-01-05,1,5,4997,635,1,0,0,1,c,a,...,0,NaN,NaN,NaN,2013,1,5,5,1,1


In [66]:
train_df.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'Year', 'Month', 'Day', 'weekday',
       'weekofyear', 'weekend'],
      dtype='object')

In [70]:
test_df.drop('weekday',inplace=True,axis=1)
test_df.drop('weekofyear',inplace=True,axis=1)
test_df.drop('weekend',inplace=True,axis=1)

In [68]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41088 entries, 2015-08-01 to 2015-09-17
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Id                         41088 non-null  int64  
 1   Store                      41088 non-null  int64  
 2   DayOfWeek                  41088 non-null  int64  
 3   Open                       41077 non-null  float64
 4   Promo                      41088 non-null  int64  
 5   StateHoliday               41088 non-null  object 
 6   SchoolHoliday              41088 non-null  int64  
 7   StoreType                  41088 non-null  object 
 8   Assortment                 41088 non-null  object 
 9   CompetitionDistance        40992 non-null  float64
 10  CompetitionOpenSinceMonth  25872 non-null  float64
 11  CompetitionOpenSinceYear   25872 non-null  float64
 12  Promo2                     41088 non-null  int64  
 13  Promo2SinceWeek            23856 non-

In [67]:
test_df.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,weekday,weekofyear,weekend
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-01,40233,1,5,1.0,0,0,1,c,a,1270.0,...,0,NaN,NaN,NaN,2015,8,1,5,31,1
2015-08-02,39377,1,6,0.0,0,0,1,c,a,1270.0,...,0,NaN,NaN,NaN,2015,8,2,6,31,1
2015-08-03,38521,1,0,1.0,1,0,1,c,a,1270.0,...,0,NaN,NaN,NaN,2015,8,3,0,32,1
2015-08-04,37665,1,1,1.0,1,0,1,c,a,1270.0,...,0,NaN,NaN,NaN,2015,8,4,1,32,0
2015-08-05,36809,1,2,1.0,1,0,1,c,a,1270.0,...,0,NaN,NaN,NaN,2015,8,5,2,32,0


In [46]:
test_df.describe()

,Unnamed: 0,Store,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,weekday,weekofyear,weekend
count,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.0,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000
mean,20543.500000,555.899533,2.979167,0.854094,0.395833,0.443487,5082.359813,7.022196,2009.144860,0.580607,23.408879,2011.896028,2015.0,8.354167,13.520833,2.979167,34.645833,0.437500
std,11861.228267,320.274496,2.015481,0.353017,0.489035,0.496802,7218.187006,2.494084,5.484756,0.493466,10.856721,1.292403,0.0,0.478266,8.448450,2.015481,2.015481,0.496084
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000,2015.0,8.000000,1.000000,0.000000,31.000000,0.000000
25%,10271.750000,279.750000,1.000000,1.000000,0.000000,0.000000,720.000000,6.000000,2008.000000,0.000000,18.000000,2011.000000,2015.0,8.000000,6.750000,1.000000,33.000000,0.000000
50%,20543.500000,553.500000,3.000000,1.000000,0.000000,0.000000,2425.000000,7.000000,2010.000000,1.000000,22.000000,2012.000000,2015.0,8.000000,12.500000,3.000000,35.000000,0.000000
75%,30815.250000,832.250000,5.000000,1.000000,1.000000,1.000000,6435.000000,9.000000,2011.000000,1.000000,31.000000,2012.000000,2015.0,9.000000,19.250000,5.000000,36.000000,1.000000
max,41087.000000,1115.000000,6.000000,1.000000,1.000000,1.000000,75860.000000,12.000000,2015.000000,1.000000,49.000000,2015.000000,2015.0,9.000000,31.000000,6.000000,38.000000,1.000000


In [47]:
train_df.columns

Index(['Unnamed: 0', 'Store', 'DayOfWeek', 'Sales', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Day', 'weekday',
       'weekofyear', 'weekend', 'PromoInterval', 'Assortment', 'StoreType'],
      dtype='object')